In [21]:
import numpy as np
import xarray as xr
from pyrte_rrtmgp.rrtmgp import GasOptics
from pyrte_rrtmgp.rrtmgp_data_files import GasOpticsFiles
import myutils.data_helper as dh
import moist_thermodynamics.constants as mtc
import angles


def make_atmosphere(p, T, h2o_vmr, T_s=None):
    """Create a pyRTE-RRTMG atmosphere from pressure, temperature and humidity arrays."""
    if np.any(p[..., 0] < p[..., -1]):
        raise ValueError("Arrays need to be passed in ascending order")

    if T_s is None:
        T_s = T[..., 0]

    atmosphere = xr.Dataset(
        data_vars={
            "pres_level": (("column", "level"), p),
            "temp_level": (("column", "level"), T),
            "pres_layer": (("column", "layer"), 0.5 * (p[..., 1:] + p[..., :-1])),
            "temp_layer": (("column", "layer"), 0.5 * (T[..., 1:] + T[..., :-1])),
            "surface_temperature": (("column",), T_s),
            "h2o": (("column", "layer"), 0.5 * (h2o_vmr[..., 1:] + h2o_vmr[..., :-1])),
            "o3": 0,
            "co2": 348e-6,
            "ch4": 1650e-9,
            "n2o": 306e-9,
            "n2": 0.7808,
            "o2": 0.2095,
            "co": 0.0,
        },
    )

    return atmosphere

In [2]:
ds = xr.open_dataset(
    "ipfs://bafybeiesyutuduzqwvu4ydn7ktihjljicywxeth6wtgd5zi4ynxzqngx4m", engine="zarr"
)
ds = ds.pipe(dh.interpolate_gaps).pipe(dh.extrapolate_sfc)
ds = ds.isel(  # pick every 10th sonde
    altitude=slice(0, 1100),  # skip upper-most layers due to NaN values
).dropna(dim="sonde", how="any", subset=["ta", "q", "p"])
ds = ds.assign(
    mu0=xr.apply_ufunc(
        angles.cos_zenith_angle,
        ds.launch_time,
        ds.launch_lat,
        ds.launch_lon,
        vectorize=True,
    )
)
atmosphere = make_atmosphere(ds.p.values, ds.ta.values, ds.q.values)
atmosphere

<xarray.Dataset> Size: 22MB
Dimensions:              (column: 989, level: 1100, layer: 1099)
Dimensions without coordinates: column, level, layer
Data variables: (12/13)
    pres_level           (column, level) float32 4MB 1.012e+05 ... 2.501e+04
    temp_level           (column, level) float32 4MB 296.4 296.3 ... 234.1 234.1
    pres_layer           (column, layer) float32 4MB 1.011e+05 ... 2.503e+04
    temp_layer           (column, layer) float32 4MB 296.3 296.3 ... 234.2 234.1
    surface_temperature  (column) float32 4kB 296.4 298.6 300.6 ... 302.6 302.6
    h2o                  (column, layer) float32 4MB 0.01527 ... 8.613e-05
    ...                   ...
    co2                  float64 8B 0.000348
    ch4                  float64 8B 1.65e-06
    n2o                  float64 8B 3.06e-07
    n2                   float64 8B 0.7808
    o2                   float64 8B 0.2095
    co                   float64 8B 0.0

In [3]:
gas_optics_lw = GasOptics(gas_optics_file=GasOpticsFiles.LW_G256)
gas_optics_sw = GasOptics(gas_optics_file=GasOpticsFiles.SW_G224)
optical_props = gas_optics_lw.compute(atmosphere, add_to_input=False)

optical_props = optical_props.assign(surface_emissivity=0.98)

In [ ]:
clr_fluxes = optical_props.rte.solve(add_to_input=False)

In [ ]:
optical_props_sw = gas_optics_sw.compute(atmosphere, add_to_input=False)

In [ ]:
optical_props_sw["surface_albedo"] = 0.06
optical_props_sw = optical_props_sw.assign(mu0=("column", ds.mu0.values))
sw_fluxes = optical_props_sw.rte.solve(add_to_input=False)

In [16]:
sw_fluxes

<xarray.Dataset> Size: 26MB
Dimensions:       (column: 989, level: 1100)
Coordinates:
  * column        (column) int64 8kB 0 1 2 3 4 5 6 ... 983 984 985 986 987 988
Dimensions without coordinates: level
Data variables:
    sw_flux_up    (column, level) float64 9MB 62.29 62.32 62.35 ... 80.82 80.84
    sw_flux_down  (column, level) float64 9MB 1.038e+03 1.039e+03 ... 871.4
    sw_flux_dir   (column, level) float64 9MB 988.7 989.1 989.6 ... 870.6 871.4

In [ ]:
mtc.gravity_earth / mtc.isobari(sw_fluxes.sw_flux_up - sw_fluxes.sw_flux_down).diff(
    "level"
).isel(level=slice(0, -1)) / atmosphere.pres_layer.diff("layer").rename(
    {"layer": "level"}
)

<xarray.DataArray (column: 989, level: 1098)> Size: 9MB
array([[0.00352248, 0.00350861, 0.00350855, ..., 0.00721503, 0.00848838,
        0.01031889],
       [0.00267321, 0.00268035, 0.0029021 , ..., 0.00777779, 0.0090337 ,
        0.01115345],
       [0.00321026, 0.00283637, 0.00278604, ..., 0.01286241, 0.01558763,
        0.01930246],
       ...,
       [0.00225113, 0.00196631, 0.00211465, ..., 0.00698952, 0.00833897,
        0.01060139],
       [0.00209028, 0.00206548, 0.00213276, ..., 0.00730598, 0.00850001,
        0.01049558],
       [0.00198541, 0.00197311, 0.00195649, ..., 0.00777437, 0.00895224,
        0.01134663]], shape=(989, 1098))
Coordinates:
  * column   (column) int64 8kB 0 1 2 3 4 5 6 7 ... 982 983 984 985 986 987 988
Dimensions without coordinates: level

In [25]:
mtc.cpd

1004.700406965869